In [1]:
from PIL import Image
from ultralytics import RTDETR
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
import json
import glob
%matplotlib inline

In [2]:
# Check if CUDA (GPU) is available and set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0") # Use the first GPU
    print(f"Training on GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA not available. Training on CPU.")

Training on GPU: NVIDIA GeForce RTX 2060 SUPER


In [3]:
torch.cuda.empty_cache()

I modified the `rtdetr-shrimp-s.yaml` file to make the model lightweight by reducing its overall complexity and computational requirements. Here's a breakdown of the specific modifications and the reasoning behind them:

1. __`scale` parameter modification:__

   - __Original:__ `s: [0.33, 0.25, 256]`

   - __Modified:__ `s: [0.17, 0.125, 128]`

   - __Why:__ The `scale` parameter controls the compound scaling constants for the model, specifically affecting its depth, width, and maximum number of channels.

     - Reducing the first value (depth from 0.33 to 0.17) makes the model shallower by decreasing the number of layers or blocks.
     - Reducing the second value (width from 0.25 to 0.125) makes the model narrower by decreasing the number of channels in each layer.
     - Reducing the third value (max_channels from 256 to 128) sets a lower upper limit on the number of channels.
     - These changes collectively reduce the total number of parameters and computational operations, making the model significantly lighter.

2. __Channel size reductions in `backbone` and `head` modules:__

   - I systematically reduced the channel arguments for various modules throughout the `backbone` and `head` sections. For instance:

     - `HGStem, [8, 12]` was changed to `HGStem, [4, 6]`
     - `HGBlock, [12, 32, 3]` was changed to `HGBlock, [6, 16, 3]`
     - `DWConv, [32, 3, 2, 1, False]` was changed to `DWConv, [16, 3, 2, 1, False]`
     - `Conv, [32, 1, 1, None, 1, 1, False]` was changed to `Conv, [16, 1, 1, None, 1, 1, False]`
     - `AIFI, [128, 8]` was changed to `AIFI, [64, 8]`
     - `RepC3, [32]` was changed to `RepC3, [16]`

   - __Why:__ These arguments define the number of input and output channels for the convolutional and block modules. By reducing these channel counts (often by half), the model processes less information at each layer. This significantly decreases the number of floating-point operations (FLOPs) and the memory footprint, resulting in a faster and more resource-efficient model suitable for training in environments with limited computational resources.


In [6]:
model = RTDETR('E:/PD1ModelTrainings/PD1ModelTrainingCodes/rtdetr-shrimp-s.yaml')

In [ ]:
result = model.train(
    data='Shrimp-larvae-detection-1/data.yaml',
    epochs = 60,
    batch = 12,
    device=0
)

Ultralytics 8.3.204  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Shrimp-larvae-detection-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=E:/PD1ModelTrainings/PD1ModelTrainingCodes/rtdetr-shrimp-s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train13, nbs=64, nms=False, opset=None, optimize=False, optimiz

e:\PD1ModelTrainings\DLenv\Lib\site-packages\torch\autograd\graph.py:824: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:97.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/60      11.4G      1.556     0.3629     0.3614        319        640: 100% ━━━━━━━━━━━━ 345/345 1.3it/s 4:27<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 3.9it/s 4.3s0.2s
                   all        386      13790      0.362      0.609      0.397      0.196

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       2/60      4.49G     0.9248      0.543     0.2577        653        640: 0% ──────────── 0/345  0.6s

e:\PD1ModelTrainings\DLenv\Lib\site-packages\torch\autograd\graph.py:824: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:97.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/60      7.68G     0.9908     0.4788     0.1677        463        640: 100% ━━━━━━━━━━━━ 345/345 0.5it/s 12:08<1.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 3.7it/s 4.6s0.3s
                   all        386      13790       0.69      0.764      0.695      0.293

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       3/60      4.38G     0.7734     0.4723     0.1035        299        640: 0% ──────────── 0/345  0.5s

...

      41/60      8.84G     0.4966     0.4328    0.05804        290        640: 100% ━━━━━━━━━━━━ 345/345 1.6it/s 3:31<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 3.8it/s 4.5s0.3s
                   all        386      13790       0.87      0.909      0.927      0.553

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      42/60       4.8G     0.5317      0.433    0.05927        606        640: 0% ──────────── 0/345  0.6s

e:\PD1ModelTrainings\DLenv\Lib\site-packages\torch\autograd\graph.py:824: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:97.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      42/60      7.56G      0.492     0.4322    0.05815        647        640: 100% ━━━━━━━━━━━━ 345/345 1.2it/s 4:44<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 3.7it/s 4.5s0.3s
                   all        386      13790       0.87      0.914      0.928      0.557

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      43/60      4.59G     0.4667     0.4258    0.04441        580        640: 0% ──────────── 0/345  0.6s

e:\PD1ModelTrainings\DLenv\Lib\site-packages\torch\autograd\graph.py:824: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:97.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      43/60      10.5G     0.4848     0.4296    0.05803        984        640: 34% ━━━━──────── 118/345 1.4it/s 1:22<2:41